In [1]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

# Memory management
import gc 
import sys

#utils contain all utility functions agg_categorical , return_size , agg_numeric
#from utils import *

In [20]:
def agg_numeric(df, group_var, df_name):
    """Aggregates the numeric values in a dataframe. This can
    be used to create features for each instance of the grouping variable.
    
    Parameters
    --------
        df (dataframe): 
            the dataframe to calculate the statistics on
        group_var (string): 
            the variable by which to group df
        df_name (string): 
            the variable used to rename the columns
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated for 
            all numeric columns. Each instance of the grouping variable will have 
            the statistics (mean, min, max, sum; currently supported) calculated. 
            The columns are also renamed to keep track of features created.
    
    """
    # Remove id variables other than grouping variable
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids

    # Group by the specified variable and calculate the statistics
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()

    # Need to create new column names
    columns = [group_var]

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))

    agg.columns = columns
    return agg


def count_categorical(df, group_var, df_name):
    """Computes counts and normalized counts for each observation
    of `group_var` of each unique category in every categorical variable
    
    Parameters
    --------
    df : dataframe 
        The dataframe to calculate the value counts for.
        
    group_var : string
        The variable by which to group the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    df_name : string
        Variable added to the front of column names to keep track of columns

    
    Return
    --------
    categorical : dataframe
        A dataframe with counts and normalized counts of each unique category in every categorical variable
        with one row for every unique value of the `group_var`.
        
    """
    
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # Make sure to put the identifying id on the column
    categorical[group_var] = df[group_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
    
    column_names = [group_var]
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['count', 'count_norm']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical


#Plots the disribution of a variable colored by value of the target
def kde_target(var_name, df):
    
    # Calculate the correlation coefficient between the new variable and the target
    corr = df['TARGET'].corr(df[var_name])
    
    # Calculate medians for repaid vs not repaid
    avg_repaid = df.ix[df['TARGET'] == 0, var_name].median()
    avg_not_repaid = df.ix[df['TARGET'] == 1, var_name].median()
    
    plt.figure(figsize = (12, 6))
    
    # Plot the distribution for target == 0 and target == 1
    sns.kdeplot(df.ix[df['TARGET'] == 0, var_name], label = 'TARGET == 0')
    sns.kdeplot(df.ix[df['TARGET'] == 1, var_name], label = 'TARGET == 1')
    
    # label the plot
    plt.xlabel(var_name); plt.ylabel('Density'); plt.title('%s Distribution' % var_name)
    plt.legend();
    
    # print out the correlation
    print('The correlation between %s and the TARGET is %0.4f' % (var_name, corr))
    # Print out average values
    print('Median value for loan that was not repaid = %0.4f' % avg_not_repaid)
    print('Median value for loan that was repaid =     %0.4f' % avg_repaid)


def return_size(df):
    """Return size of dataframe in gigabytes"""
    return round(sys.getsizeof(df) / 1e9, 2)


#Function to Convert Data Types


def convert_types(df, print_info = False):
    
    original_memory = df.memory_usage().sum()
    
    # Iterate through each column
    for c in df:
        
        # Convert ids and booleans to integers
        if ('SK_ID' in c):
            df[c] = df[c].fillna(0).astype(np.int32)
            
        # Convert objects to category
        elif (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
            df[c] = df[c].astype('category')
        
        # Booleans mapped to integers
        elif list(df[c].unique()) == [1, 0]:
            df[c] = df[c].astype(bool)
        
        # Float64 to float32
        elif df[c].dtype == float:
            df[c] = df[c].astype(np.float32)
            
        # Int64 to int32
        elif df[c].dtype == int:
            df[c] = df[c].astype(np.int32)
        
    new_memory = df.memory_usage().sum()
    
    if print_info:
        print(f'Original Memory Usage: {round(original_memory / 1e9, 2)} gb.')
        print(f'New Memory Usage: {round(new_memory / 1e9, 2)} gb.')
        
    return df


In [3]:
train = pd.read_csv('data/app_train_processed.csv')
test = pd.read_csv('data/app_test_processed.csv')

In [4]:
print(return_size(train))
print(return_size(test))



0.59
0.09


In [41]:
previous = pd.read_csv('data/previous_application.csv')
previous = convert_types(previous, print_info=True)


Original Memory Usage: 0.49 gb.
New Memory Usage: 0.16 gb.


In [42]:
# Calculate aggregate statistics for each numeric column
print('prior Previous aggregation shape: ', previous.shape)

previous_agg = agg_numeric(previous, 'SK_ID_CURR', 'previous')
print('Previous aggregation shape: ', previous_agg.shape)


prior Previous aggregation shape:  (1670214, 37)
Previous aggregation shape:  (338857, 91)


In [43]:
# Calculate value counts for each categorical column
previous_counts = agg_categorical(previous, 'SK_ID_CURR', 'previous')
print('Previous counts shape: ', previous_counts.shape)

Previous counts shape:  (338857, 285)


In [12]:
train = convert_types(train)

In [13]:
test = convert_types(test)

In [45]:
previous_counts.reset_index(inplace=True)

In [46]:
previous_counts.merge(previous_agg,on ='SK_ID_CURR', how = 'left')

,SK_ID_CURR,previous_NAME_GOODS_CATEGORY_Animals_mean,previous_NAME_GOODS_CATEGORY_Animals_sum,previous_NAME_GOODS_CATEGORY_House Construction_mean,previous_NAME_GOODS_CATEGORY_House Construction_sum,previous_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal_mean,previous_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal_sum,previous_NAME_CASH_LOAN_PURPOSE_Money for a third person_mean,previous_NAME_CASH_LOAN_PURPOSE_Money for a third person_sum,previous_NAME_CASH_LOAN_PURPOSE_Hobby_mean,...,previous_DAYS_TERMINATION_count,previous_DAYS_TERMINATION_mean,previous_DAYS_TERMINATION_max,previous_DAYS_TERMINATION_min,previous_DAYS_TERMINATION_sum,previous_NFLAG_INSURED_ON_APPROVAL_count,previous_NFLAG_INSURED_ON_APPROVAL_mean,previous_NFLAG_INSURED_ON_APPROVAL_max,previous_NFLAG_INSURED_ON_APPROVAL_min,previous_NFLAG_INSURED_ON_APPROVAL_sum
0,100001,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,1,-1612.000000,-1612.0,-1612.0,-1612.0,1,0.000000,0.0,0.0,0.0
1,100002,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,1,-17.000000,-17.0,-17.0,-17.0,1,0.000000,0.0,0.0,0.0
2,100003,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,3,-1047.333374,-527.0,-1976.0,-3142.0,3,0.666667,1.0,0.0,2.0
3,100004,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,1,-714.000000,-714.0,-714.0,-714.0,1,0.000000,0.0,0.0,0.0
4,100005,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,1,-460.000000,-460.0,-460.0,-460.0,1,0.000000,0.0,0.0,0.0
5,100006,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,4,182481.750000,365243.0,-416.0,729927.0,4,0.000000,0.0,0.0,0.0
6,100007,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,5,72143.796875,365243.0,-2041.0,360719.0,5,0.600000,1.0,0.0,3.0
7,100008,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,4,-872.750000,-66.0,-2334.0,-3491.0,4,0.250000,1.0,0.0,1.0
8,100009,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,7,51672.855469,365243.0,-1323.0,361710.0,7,0.000000,0.0,0.0,0.0
9,100010,0.0,0,0.0,0,0.0,0,0.0,0,0.0,...,1,-762.000000,-762.0,-762.0,-762.0,1,0.000000,0.0,0.0,0.0


In [48]:
previous_counts.to_csv('data/merged/prev_application_merged.csv',index=False)

In [27]:
# Merge in the previous information
train = train.merge(previous_counts, on ='SK_ID_CURR', how = 'left')


In [28]:
train = train.merge(previous_agg, on = 'SK_ID_CURR', how = 'left')

In [29]:
test = test.merge(previous_counts, on ='SK_ID_CURR', how = 'left')
test = test.merge(previous_agg, on = 'SK_ID_CURR', how = 'left')

# Remove variables to free memory
gc.enable()
del previous, previous_agg, previous_counts
gc.collect()

291

In [35]:
train.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,previous_DAYS_TERMINATION_count,previous_DAYS_TERMINATION_mean,previous_DAYS_TERMINATION_max,previous_DAYS_TERMINATION_min,previous_DAYS_TERMINATION_sum,previous_NFLAG_INSURED_ON_APPROVAL_count,previous_NFLAG_INSURED_ON_APPROVAL_mean,previous_NFLAG_INSURED_ON_APPROVAL_max,previous_NFLAG_INSURED_ON_APPROVAL_min,previous_NFLAG_INSURED_ON_APPROVAL_sum
0,100002,0,0,True,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,1.0,-17.000000,-17.0,-17.0,-17.0,1.0,0.000000,0.0,0.0,0.0
1,100003,0,0,False,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,3.0,-1047.333374,-527.0,-1976.0,-3142.0,3.0,0.666667,1.0,0.0,2.0
2,100004,1,1,True,0,67500.0,135000.0,6750.0,135000.0,0.010032,...,1.0,-714.000000,-714.0,-714.0,-714.0,1.0,0.000000,0.0,0.0,0.0
3,100006,0,0,True,0,135000.0,312682.5,29686.5,297000.0,0.008019,...,4.0,182481.750000,365243.0,-416.0,729927.0,4.0,0.000000,0.0,0.0,0.0
4,100007,0,0,True,0,121500.0,513000.0,21865.5,513000.0,0.028663,...,5.0,72143.796875,365243.0,-2041.0,360719.0,5.0,0.600000,1.0,0.0,3.0


In [34]:
test.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,previous_DAYS_TERMINATION_count,previous_DAYS_TERMINATION_mean,previous_DAYS_TERMINATION_max,previous_DAYS_TERMINATION_min,previous_DAYS_TERMINATION_sum,previous_NFLAG_INSURED_ON_APPROVAL_count,previous_NFLAG_INSURED_ON_APPROVAL_mean,previous_NFLAG_INSURED_ON_APPROVAL_max,previous_NFLAG_INSURED_ON_APPROVAL_min,previous_NFLAG_INSURED_ON_APPROVAL_sum
0,100001,0,0,True,0,135000.0,568800.0,20560.5,450000.0,0.018850,...,1.0,-1612.000000,-1612.0,-1612.0,-1612.0,1.0,0.000000,0.0,0.0,0.0
1,100005,0,0,True,0,99000.0,222768.0,17370.0,180000.0,0.035792,...,1.0,-460.000000,-460.0,-460.0,-460.0,1.0,0.000000,0.0,0.0,0.0
2,100013,0,1,True,0,202500.0,663264.0,69777.0,630000.0,0.019101,...,3.0,-710.333313,-85.0,-1702.0,-2131.0,3.0,0.333333,1.0,0.0,1.0
3,100028,0,0,True,2,315000.0,1575000.0,49018.5,1575000.0,0.026392,...,3.0,121182.664062,365243.0,-1081.0,363548.0,3.0,0.000000,0.0,0.0,0.0
4,100038,0,1,False,1,180000.0,625500.0,32067.0,625500.0,0.010032,...,1.0,-449.000000,-449.0,-449.0,-449.0,1.0,0.000000,0.0,0.0,0.0


In [37]:
train.to_csv('data/merged/train_prev_app_merged.csv',index=False)

In [38]:
test.to_csv('data/merged/test_prev_app_merged.csv',index=False)

In [40]:
# Remove variables to free memory
gc.enable()
del train,test
gc.collect()

5930